In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import time

In [2]:
#setup for splinter
executable_path = {'executable_path': 'c:/bin/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.britannica.com/topic/list-of-state-capitals-in-the-United-States-2119210'
browser.visit(url)
tables = pd.read_html(url)
state_capitals = tables[0].copy()
capital_list = state_capitals["capital"].to_list()

In [3]:
# Craigslist data extractopm
#using request and soup
CL_url = "https://minneapolis.craigslist.org/d/cars-trucks/search/cta"
CL_response = requests.get(CL_url)
time.sleep(2)
# Create BeautifulSoup object; parse with 'html.parser'
CL_soup = BeautifulSoup(CL_response.text, 'lxml')
#print(CL_soup)

In [6]:
model_list = ["subaru","honda", "toyota","BMW","mercedes","ford","dodge", "chrysler","chevrolet","chevy"]
city_list = capital_list
model_df = pd.DataFrame()
model_df["Make"]=model_list

for city in city_list:
    
    model_counter = []

    for model in model_list:
        print(f" {city} and {model}")
        CL_url = f"https://{city}.craigslist.org/search/cta?auto_make_model={model}&min_auto_year=2010&max_auto_year=2015"
        CL_response = requests.get(CL_url)
        # Create BeautifulSoup object; parse with 'html.parser'
        CL_soup = BeautifulSoup(CL_response.text, 'lxml')
        CL_results = CL_soup.find_all('span', class_="totalcount")
        print(CL_results)
        if CL_results == "":
            counter = 0
        else:
            counter = CL_results[0].text
        model_counter.append(int(counter))
        
    model_df[city]=model_counter

SyntaxError: invalid syntax (<ipython-input-6-fbe32aeeed0d>, line 20)

In [ ]:
final_df = model_df.transpose(copy = True)